In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

In [2]:
ratings = pd.read_csv("C:\\share\\data\\ml-1m\\ratings.dat",
                      sep="::",
                      header=None,
                      engine='python',
                      names=["UserID", "MovieID", "Rating", "Timestamp"])
ratings.dropna(axis=0, inplace=True)
print(ratings.shape)
ratings[:10]

(1000209, 4)


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [3]:
# 使用mini数据集进行逻辑实现
ratings = pd.read_csv("../temp/ratings_mini.csv")
ratings

,UserID,MovieID,Rating,Timestamp
0,10,648,4,978224925
1,10,2628,3,978228408
2,10,1682,5,978226319
3,10,2054,4,978229409
4,10,1252,3,979775409
...,...,...,...,...
112297,6040,2028,5,956704519
112298,6040,1089,4,956704996
112299,6040,1090,3,956715518
112300,6040,1094,5,956704887


In [4]:
ratings2 = pd.DataFrame([[0]], columns=['temp'])
movie_names = set()
for (idx, row), idxlimit in zip(ratings.iterrows(), range(10000000)):
    movie_names.add(row['MovieID'])
movie_names_sort = sorted(list(movie_names))
ratings2 = pd.DataFrame([], columns=movie_names_sort)
ratings2.shape

(0, 207)

In [5]:
user_names = set()
for (idx, row), idxlimit in zip(ratings.iterrows(), range(100000000)):
    user_names.add(row['UserID'])
user_names_sort = sorted(list(user_names))
user_names_pd = pd.DataFrame(user_names_sort, columns=['UserID'])
ratings2['UserID'] = user_names_pd['UserID']
ratings2.fillna(3, inplace=True)
ratings2.shape

C:\Users\wangy\AppData\Local\Temp\ipykernel_12744\4283252115.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings2.fillna(3, inplace=True)


(839, 208)

In [6]:
ratings3 = ratings2.copy()
ratings3.index = ratings3['UserID']
ratings3.drop(columns=['UserID'], inplace=True)
for (idx, row), idxlimit in zip(ratings.iterrows(), range(100000000)):
    ratings3.loc[row['UserID'], row['MovieID']] = row['Rating']
ratings3 = (ratings3 - 1) / 4
ratings3

,1,11,21,32,34,39,47,50,110,111,...,3527,3578,3623,3671,3702,3703,3751,3753,3755,3793
UserID,,,,,,,,,,,,,,,,,,,,,
10,1.00,0.50,0.50,1.00,0.50,0.50,0.50,0.50,0.75,0.50,...,0.50,0.50,0.5,0.75,0.50,0.25,1.00,0.50,0.5,0.50
23,0.75,0.25,0.75,0.75,0.50,0.75,0.25,0.75,0.50,0.50,...,0.75,0.75,0.5,0.50,0.00,0.25,0.50,0.50,0.5,0.75
33,0.50,0.50,0.50,0.25,0.25,0.50,1.00,0.50,0.25,1.00,...,1.00,0.50,0.5,0.50,0.50,0.50,0.50,0.50,0.5,0.50
36,1.00,0.75,1.00,1.00,1.00,0.50,1.00,0.50,0.75,0.50,...,1.00,1.00,1.0,1.00,0.75,0.50,0.50,0.75,0.5,0.75
42,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,...,0.75,0.50,0.5,0.50,1.00,0.75,0.50,0.50,0.5,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,1.00,0.75,0.75,0.50,0.75,0.75,0.50,0.75,1.00,0.50,...,0.75,0.50,0.5,0.50,0.50,0.50,0.50,0.50,0.5,0.50
6011,1.00,0.50,0.50,0.50,1.00,0.50,0.50,0.50,1.00,0.50,...,0.25,0.50,0.5,0.50,0.50,0.50,0.50,0.50,0.5,0.50
6016,0.75,0.50,0.75,0.75,0.75,0.75,0.75,0.50,0.50,0.75,...,0.00,0.50,0.5,0.50,0.50,0.50,0.75,0.50,0.5,0.50


In [7]:
ratings3.to_csv("../temp/ratings3.csv")

In [8]:
from sklearn.decomposition import TruncatedSVD

model = TruncatedSVD(n_components=30)
model.fit(ratings3)
ratings3_svd = model.transform(ratings3)
ratings3_svd

array([[ 9.96161906e+00, -1.11559515e+00,  4.85052499e-01, ...,
         1.75093635e-01, -3.45360007e-01, -2.29698603e-01],
       [ 8.43101023e+00, -3.86964542e-01,  3.80157499e-03, ...,
        -1.34911592e-01, -6.87355152e-03,  9.92938602e-02],
       [ 8.90307325e+00, -8.55063732e-02,  5.01378749e-01, ...,
         9.35796825e-02,  3.02808610e-01, -3.29025804e-02],
       ...,
       [ 8.49191200e+00,  8.98877873e-01,  6.24201709e-02, ...,
        -2.25469116e-01, -1.17852408e-01, -2.98406883e-01],
       [ 8.93949831e+00,  9.94210998e-01,  6.64308203e-02, ...,
        -1.27143049e-01, -5.92339800e-01, -3.30896924e-01],
       [ 8.47957762e+00,  1.13054635e+00,  3.17956847e-01, ...,
         3.61691508e-01, -9.08890439e-02,  3.60072307e-03]])

In [9]:
ratings4 = model.inverse_transform(ratings3_svd)
ratings4 = pd.DataFrame(ratings4,
                        columns=ratings3.columns,
                        index=ratings3.index)
ratings4

,1,11,21,32,34,39,47,50,110,111,...,3527,3578,3623,3671,3702,3703,3751,3753,3755,3793
UserID,,,,,,,,,,,,,,,,,,,,,
10,0.994466,0.742929,0.554493,0.793777,0.606549,0.733577,0.414224,0.536309,0.750822,0.429223,...,0.556118,0.633557,0.575902,0.769548,0.484945,0.502222,0.690482,0.548306,0.541710,0.562999
23,0.546510,0.464338,0.670094,0.709081,0.563318,0.618753,0.581580,0.682825,0.549039,0.538926,...,0.649706,0.645559,0.558385,0.437968,0.558707,0.572652,0.479756,0.499263,0.482610,0.613976
33,0.515914,0.567201,0.459160,0.475676,0.408298,0.518900,0.675342,0.634584,0.532591,0.843739,...,0.685832,0.524608,0.551037,0.562559,0.543413,0.577807,0.425486,0.638307,0.532373,0.569137
36,1.045928,0.737965,0.646390,0.738966,0.740882,0.797699,0.855785,0.849430,0.829621,0.480284,...,0.818051,0.973705,0.747142,0.813564,0.645537,0.661544,0.645075,0.728177,0.659562,0.849968
42,0.630705,0.541768,0.440733,0.656147,0.535630,0.491544,0.492925,0.516497,0.606662,0.466590,...,0.783142,0.537335,0.495370,0.544051,0.666698,0.666392,0.532887,0.462395,0.532485,0.566157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,0.829883,0.525080,0.645273,0.568368,0.624329,0.569328,0.416548,0.559313,1.050555,0.540725,...,0.773967,0.573618,0.592189,0.695581,0.686949,0.665699,0.474795,0.559270,0.500256,0.477862
6011,0.923288,0.609186,0.561288,0.628856,0.685994,0.504775,0.505843,0.569739,1.002602,0.378230,...,0.779477,0.654134,0.593110,0.728539,0.662937,0.555292,0.619483,0.675232,0.500809,0.657390
6016,0.790236,0.538741,0.646228,0.562762,0.838786,0.630998,0.507255,0.665582,0.482977,0.731102,...,0.337498,0.559290,0.529511,0.485418,0.463981,0.521027,0.674588,0.485241,0.479874,0.533298


In [10]:
# 在推荐结果中去掉已经打过分的
ratings5 = ratings4.copy()
for (idx, row) in ratings.iterrows():
    ratings5.loc[row['UserID'], row['MovieID']] = 0
ratings5

,1,11,21,32,34,39,47,50,110,111,...,3527,3578,3623,3671,3702,3703,3751,3753,3755,3793
UserID,,,,,,,,,,,,,,,,,,,,,
10,0.000000,0.742929,0.554493,0.000000,0.606549,0.733577,0.414224,0.536309,0.000000,0.429223,...,0.556118,0.633557,0.575902,0.000000,0.000000,0.000000,0.000000,0.548306,0.541710,0.562999
23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.549039,0.000000,...,0.000000,0.000000,0.558385,0.437968,0.000000,0.000000,0.000000,0.499263,0.482610,0.000000
33,0.515914,0.567201,0.459160,0.000000,0.000000,0.518900,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.524608,0.551037,0.000000,0.543413,0.577807,0.425486,0.638307,0.532373,0.569137
36,0.000000,0.000000,0.000000,0.000000,0.000000,0.797699,0.000000,0.849430,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.661544,0.645075,0.000000,0.659562,0.000000
42,0.630705,0.541768,0.440733,0.000000,0.000000,0.491544,0.492925,0.516497,0.606662,0.466590,...,0.000000,0.537335,0.000000,0.544051,0.000000,0.000000,0.532887,0.462395,0.532485,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,0.000000,0.000000,0.000000,0.568368,0.000000,0.000000,0.416548,0.000000,0.000000,0.540725,...,0.000000,0.573618,0.592189,0.695581,0.686949,0.665699,0.474795,0.559270,0.500256,0.477862
6011,0.000000,0.000000,0.561288,0.628856,0.000000,0.504775,0.505843,0.569739,0.000000,0.378230,...,0.000000,0.654134,0.593110,0.728539,0.662937,0.555292,0.619483,0.675232,0.500809,0.657390
6016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.665582,0.482977,0.000000,...,0.000000,0.559290,0.529511,0.485418,0.463981,0.521027,0.000000,0.485241,0.479874,0.533298


In [11]:
result_pd = pd.DataFrame([], columns=['rec'], index=ratings3.index)
test_data = None
for (idx, row), (idx2, row2), i in zip(ratings4.iterrows(), ratings3.iterrows(), range(1000)):
    # print(row[row2==0.5].sort_values(ascending=False))
    test_data = row[row2 == 0.5].sort_values(ascending=False)
    result_pd.loc[idx, 'rec'] = ",".join(
        [str(iii) + ":" + str(rrr)[:5] for iii, rrr, jjj in zip(test_data.index, test_data, range(10))])
result_pd

,rec
UserID,
10,"1396:0.825,924:0.774,1968:0.758,2987:0.753,240..."
23,"1270:0.806,2571:0.772,1200:0.753,1214:0.750,11..."
33,"457:0.919,1036:0.811,296:0.796,541:0.760,1090:..."
36,"50:0.849,2321:0.837,318:0.798,39:0.797,367:0.7..."
42,"2028:0.748,2571:0.742,2716:0.729,1653:0.720,21..."
...,...
6010,"733:0.776,318:0.762,2640:0.760,480:0.744,1073:..."
6011,"733:0.946,380:0.841,2716:0.783,1356:0.764,1127..."
6016,"912:0.798,3160:0.780,1213:0.732,1234:0.732,913..."


In [12]:
result_pd.to_csv("../temp/result.csv")